<a href="https://colab.research.google.com/github/muhammed-saeed/Naija-Pidgin/blob/main/Entire_Bible_PCM_EN_base_line_Translation_model_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup Libraries and Dataset


In [2]:
# ! git clone --recursive https://github.com/multi30k/dataset.git multi30k-dataset

In [3]:
# !rm -rf /content/multi30k-dataset

In [4]:
%%capture
# W and B -- For Logging
! pip install wandb

# Sacremoses -- For Tokenizing
! pip install sacremoses


In [5]:

# # fairseq -- For training and evaluation of the model
# ! git clone https://github.com/pytorch/fairseq
# %cd fairseq
# ! pip install --editable ./
# %cd ..

In [6]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
# !pip install fairseq 
# 


In [8]:
!git clone https://github.com/pytorch/fairseq
%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH


Cloning into 'fairseq'...
remote: Enumerating objects: 31399, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 31399 (delta 28), reused 39 (delta 26), pack-reused 31347
Receiving objects: 100% (31399/31399), 21.68 MiB | 28.57 MiB/s, done.
Resolving deltas: 100% (23100/23100), done.
/content/fairseq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 92 kB 7.4 MB/s 
     |████████████████████████████████| 123 kB 68.0 MB/s 
     |████████████████████████████████| 236 kB 68.9 MB/s 
     |████████████████████████████████| 112 kB 72.7 MB/s 
     |████████████████████████████████| 596 kB 64.1 MB/s 
  Created wheel for antlr4-python3-runtime: 

In [9]:
!git clone https://github.com/facebookresearch/xformers.git


%cd xformers
!pip install -r requirements.txt
!pip install -e .


Cloning into 'xformers'...
remote: Enumerating objects: 4769, done.
remote: Counting objects: 100% (473/473), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 4769 (delta 254), reused 414 (delta 234), pack-reused 4296
Receiving objects: 100% (4769/4769), 15.94 MiB | 27.62 MiB/s, done.
Resolving deltas: 100% (3024/3024), done.
/content/xformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/xformers
  Running setup.py develop for xformers


# Note

**then created folder called it en_pd_data
inside this folder I have created three. sub-folders (train, test, val)

inside each I have renamed the en-pd parrallel dataset likes follows train.lc.norm.tok.en / train.lc.norm.tok.pd / test.lc.norm.tok.en / val.lc.norm.tok.pd**


In [10]:
!rm -rf "/content/drive/MyDrive/Naijia Project/parrellel_data/pcm_en_1.tokenized.pcm-en"

In [11]:
!rm -rf "/content/drive/MyDrive/Naijia Project/parrellel_data/fairseq_output"

In [12]:
!rm -rf "/content/naija/pcm_en_1.tokenized.pcm-en"

In [13]:
!rm -rf "/content/drive/MyDrive/Naijia Project/parrellel_data/output/pcm_en_1.tokenized.pcm-en/"

## Pre-process and Binarize to build Vocabularies

In [14]:
# !fairseq-preprocess --source-lang pcm --target-lang en \
#   --trainpref "/content/drive/MyDrive/Naijia Project/parrellel_data/train/train.lc.norm.tok" \
#   --validpref "/content/drive/MyDrive/Naijia Project/parrellel_data/val/val.lc.norm.tok" \
#   --testpref "/content/drive/MyDrive/Naijia Project/parrellel_data/test/test.lc.norm.tok" \
#   --destdir "/content/drive/MyDrive/Naijia Project/parrellel_data/pcm_en_1.tokenized.pcm-en"  \
#   --thresholdsrc 2 \
#   --thresholdtgt 2

In [29]:
!fairseq-preprocess --source-lang pcm --target-lang en \
  --trainpref "/content/drive/MyDrive/Naijia Project/parrellel_data/train/train.lc.norm.tok" \
  --validpref "/content/drive/MyDrive/Naijia Project/parrellel_data/val/val.lc.norm.tok" \
  --testpref "/content/drive/MyDrive/Naijia Project/parrellel_data/test/test.lc.norm.tok" \
  --destdir "/content/drive/MyDrive/Naijia Project/parrellel_data/output/pcm_en_1.tokenized.pcm-en"  \
  --thresholdsrc 0 \
  --thresholdtgt 0 \
 
  # the threshold appears more than the threshold it will be counted in the vocabulary otherwise, it will be counted as unk

Error No module named 'triton'
Error No module named 'triton'


In [16]:
!export LANG=C.UTF-8
!export PYTHONIOENCODING=UTF-8

In [17]:
# data = ""
# with open("/content/pcm_parrellel.txt", "r") as fb:
#   data = fb.readlines()
# print(len(data))

# Fairseq Needs the validpref and the testpref to be filled

In [18]:
!pip install md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 6.1 MB/s 
     |████████████████████████████████| 48 kB 4.6 MB/s 
  Created wheel for md: filename=md-0.70-py3-none-any.whl size=30918 sha256=f7773a2fb2a00129def691d7fd8dc5bb406a887b10edeabb0987e75e9f337a35
  Stored in directory: /root/.cache/pip/wheels/ad/7e/ef/eea372024df797ad52b2ab91e5ea584493ebf593cada4c043b
Successfully built md


# Train the model

In [19]:
# ! fairseq-train  "/content/drive/MyDrive/Naijia Project/parrellel_data/pcm_en_1.tokenized.pcm-en" \
#   --arch transformer \
#   --dropout 0.1 \
#   --attention-dropout 0.1 \
#   --activation-dropout 0.1 \
#   --encoder-embed-dim 256 \
#   --encoder-ffn-embed-dim 512 \
#   --encoder-layers 6 \
#   --encoder-attention-heads 8 \
#   --encoder-learned-pos \
#   --decoder-embed-dim 256 \
#   --decoder-ffn-embed-dim 512 \
#   --decoder-layers 6 \
#   --decoder-attention-heads 8 \
#   --decoder-learned-pos \
#   --max-epoch 10 \
#   --optimizer adam \
#   --lr 5e-4 \
#   --batch-size 128 \
#   --seed 1 \
#   --wandb-project "PCM-EN Translation First Large Scale Corpus"

In [ ]:
! rm -rf checkpoints
! fairseq-train  "/content/drive/MyDrive/Naijia Project/parrellel_data/output/pcm_en_1.tokenized.pcm-en" \
  --arch transformer \
  --dropout 0.1 \
  --attention-dropout 0.1 \
  --encoder-embed-dim 256 \
  --encoder-ffn-embed-dim 1024 \
  --encoder-layers 5 \
  --encoder-attention-heads 2 \
  --encoder-learned-pos \
  --decoder-embed-dim 256 \
  --decoder-ffn-embed-dim 1024 \
  --decoder-layers 5 \
  --decoder-attention-heads 2 \
  --decoder-learned-pos \
  --max-epoch 50 \
  --optimizer adam \
  --lr 5e-4 \
  --batch-size 512 \
  --seed 1 \
  --encoder-layerdrop 0.0 \
  --decoder-layerdrop 0.1 \
  --criterion label_smoothed_cross_entropy \
  --activation-dropout 0.3 \
  --dropout 0.3 \
  --wandb-project "PCM-EN Translation First Large Scale Corpus"

2022-06-04 23:38:49 | WARNING | root | Triton is not available, some optimizations will not be enabled.
Error No module named 'triton'
2022-06-04 23:38:49 | WARNING | root | Triton is not available, some optimizations will not be enabled.
Error No module named 'triton'
2022-06-04 23:38:49 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-06-04 23:38:51 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'PCM-EN Translation First Large Scale Corpus', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'm

In [21]:
!nvidia-smi

Sat Jun  4 21:56:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [33]:
!nvidia-smi

Sat Jun  4 22:38:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Evaluate the model

In [39]:
# ckpt_best, beam=5
! fairseq-generate "/content/drive/MyDrive/Naijia Project/parrellel_data/output/pcm_en_1.tokenized.pcm-en"  \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 5 \
    --seed 1 \
    --scoring bleu \
    --wandb-project "PCM-EN Translation First Large Scale Corpus"

Streaming output truncated to the last 5000 lines.
T-2377	Great numbers will make attempts to get the approval of a ruler: and every man is the special friend of him who has something to give.
H-2377	-1.6515545845031738	He who gives attention to those who are in the way of profit to those who are waiting for him.
D-2377	-1.6515545845031738	He who gives attention to those who are in the way of profit to those who are waiting for him.
P-2377	-2.0126 -0.7742 -1.7869 -2.9919 -0.0049 -2.7322 -0.2271 -1.8210 -2.2921 -2.1334 -1.3943 -0.7426 -3.8384 -0.7768 -3.1086 -0.2397 -2.0202 -3.3471 -0.3932 -2.0428 -0.0027
S-436	Wetin dem wear bi like bullet proof wey dem make with iron and dia feada dey make nois like horse wey dey go fight.
T-436	And they had breastplates like iron, and the sound of their wings was as the sound of carriages, like an army of horses rushing to the fight.
H-436	-1.4941062927246094	And on the fourteenth day of the fourteenth day of the sun and the dew of the moon at the to

# Generate Average Checkpoint (Ensemble)

In [23]:
! python fairseq/scripts/average_checkpoints.py --inputs checkpoints --num-epoch-checkpoints 5 --output checkpoints/averaged.pt

python3: can't open file 'fairseq/scripts/average_checkpoints.py': [Errno 2] No such file or directory


In [24]:
# ckpt_avg, beam=5
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/averaged.pt \
    --batch-size 128 \
    --beam 5 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    --wandb-project "Multi 30K En to De Translation"

Error No module named 'triton'
Error No module named 'triton'
Traceback (most recent call last):
  File "/usr/bin/fairseq-generate", line 33, in <module>
    sys.exit(load_entry_point('fairseq', 'console_scripts', 'fairseq-generate')())
  File "/content/fairseq/fairseq_cli/generate.py", line 413, in cli_main
    main(args)
  File "/content/fairseq/fairseq_cli/generate.py", line 50, in main
    return _main(cfg, sys.stdout)
  File "/content/fairseq/fairseq_cli/generate.py", line 83, in _main
    task = tasks.setup_task(cfg.task)
  File "/content/fairseq/fairseq/tasks/__init__.py", line 46, in setup_task
    return task.setup_task(cfg, **kwargs)
  File "/content/fairseq/fairseq/tasks/translation.py", line 301, in setup_task
    cfg.source_lang, cfg.target_lang = data_utils.infer_language_pair(paths[0])
  File "/content/fairseq/fairseq/data/data_utils.py", line 30, in infer_language_pair
    for filename in PathManager.ls(path):
  File "/content/fairseq/fairseq/file_io.py", line 100, in l

In [25]:
# ckpt_avg, beam=10
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/averaged.pt \
    --batch-size 128 \
    --beam 10 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    --wandb-project "Multi 30K En to De Translation"

Error No module named 'triton'
Error No module named 'triton'
Traceback (most recent call last):
  File "/usr/bin/fairseq-generate", line 33, in <module>
    sys.exit(load_entry_point('fairseq', 'console_scripts', 'fairseq-generate')())
  File "/content/fairseq/fairseq_cli/generate.py", line 413, in cli_main
    main(args)
  File "/content/fairseq/fairseq_cli/generate.py", line 50, in main
    return _main(cfg, sys.stdout)
  File "/content/fairseq/fairseq_cli/generate.py", line 83, in _main
    task = tasks.setup_task(cfg.task)
  File "/content/fairseq/fairseq/tasks/__init__.py", line 46, in setup_task
    return task.setup_task(cfg, **kwargs)
  File "/content/fairseq/fairseq/tasks/translation.py", line 301, in setup_task
    cfg.source_lang, cfg.target_lang = data_utils.infer_language_pair(paths[0])
  File "/content/fairseq/fairseq/data/data_utils.py", line 30, in infer_language_pair
    for filename in PathManager.ls(path):
  File "/content/fairseq/fairseq/file_io.py", line 100, in l

# Export the model

In [26]:
!mkdir -p trained_model
!cp data-bin/multi30k.tokenized.de-en/dict.de.txt trained_model/dict.de.txt
! cp data-bin/multi30k.tokenized.de-en/dict.en.txt trained_model/dict.en.txt
! cp checkpoints/averaged.pt trained_model/model.pt

cp: cannot stat 'data-bin/multi30k.tokenized.de-en/dict.de.txt': No such file or directory
cp: cannot stat 'data-bin/multi30k.tokenized.de-en/dict.en.txt': No such file or directory
cp: cannot stat 'checkpoints/averaged.pt': No such file or directory


In [27]:
! fairseq-interactive \
  --path trained_model/model.pt \
  --source-lang de --target-lang en \
  --tokenizer moses \
  --task translation --cpu \
  --beam 5 \
  trained_model/

Error No module named 'triton'
Error No module named 'triton'
Traceback (most recent call last):
  File "/usr/bin/fairseq-interactive", line 33, in <module>
    sys.exit(load_entry_point('fairseq', 'console_scripts', 'fairseq-interactive')())
  File "/content/fairseq/fairseq_cli/interactive.py", line 313, in cli_main
    distributed_utils.call_main(convert_namespace_to_omegaconf(args), main)
  File "/content/fairseq/fairseq/distributed/utils.py", line 369, in call_main
    main(cfg, **kwargs)
  File "/content/fairseq/fairseq_cli/interactive.py", line 140, in main
    task = tasks.setup_task(cfg.task)
  File "/content/fairseq/fairseq/tasks/__init__.py", line 46, in setup_task
    return task.setup_task(cfg, **kwargs)
  File "/content/fairseq/fairseq/tasks/translation.py", line 309, in setup_task
    os.path.join(paths[0], "dict.{}.txt".format(cfg.source_lang))
  File "/content/fairseq/fairseq/tasks/fairseq_task.py", line 94, in load_dictionary
    return Dictionary.load(filename)
  File